Автор: Шестаков Михаил Сергеевич  
Telegram: https://t.me/mshestakov1  
email: mikhail-shestakov-2022@bk.ru

# Определение уровня сложности англоязычных фильмов

**Содержание:**  
1. [Импорт данных и общая информация](#section_1)  
 1.1. [Импорт библиотек](#section_1_1)  
 1.2. [Загрузка данных](#section_1_2)  
 1.3. [Общая информация](#section_1_3)  
    
    
2. [Предобработка данных](#section_2)  
 2.1. [Названия признаков](#section_2_1)  
 2.2. [Дубликаты](#section_2_2)  
 2.3. [Уровни английского языка](#section_2_3)  
 2.4. [Объединение датафреймов](#section_2_4)  
 2.5. [Обработка субтитров](#section_2_5)  
 
3. [Расчёт дополнительных признаков](#section_3)  
 3.1. [Признаки оценок качества текста субтитров](#section_3_1)  
 3.2. [Доли слов различного уровня](#section_3_2)  
 3.3. [Доли слов по частям речи](#section_3_3)  

4. [Обучение нейросети](#section_4)  
 4.1. [Разделение выборки на обучающую и тестовую](#section_4_1)  
 4.2. [Балансировка классов](#section_4_2)  
 4.3. [Векторизация текста](#section_4_3)  
 4.3. [Обучение](#section_4_4)  

5. [Оценка качества модели](#section_5)  
 5.1. [Оценка на тестовой выборке](#section_5_1)  
 5.2. [Матрица ошибок](#section_5_2) 

6. [Общий вывод](#section_6)  

**Цель исследования:** построить модель оценки уровня сложности фильмов на основе субтитров для изучения английского языка.  
**Инструменты:** для реализации проекта использованы модели CatBoostClassifier, LogisticRegression и RandomForestRegressor.  

**Особенности проекта:** 
Качество модели оценивается метрикой balanced accuracy.

**Исходные данные:**  
Исходные данные представлены в датасете в репозитории GitGub.
В датасете содержатся следующие признаки:  

- Movie — название фильма;
- Level — уровень знания языка;

В репозитории на GitGub представлены субтитры к соответствующим файлам.

## Импорт библиотек общая информация
<a id="section_1"></a>

### Импорт библиотек
<a id="section_1_1"></a>

In [1]:
import chardet
import difflib
import fitz
import numpy as np
import optuna
import pandas as pd
import pathlib
import pysrt
import re
import spacy
import swifter
import textstat
import torch
import transformers as ppb

from imblearn.pipeline import make_pipeline, Pipeline
from nltk.tokenize import sent_tokenize
from optuna.distributions import CategoricalDistribution as catd
from optuna.distributions import FloatDistribution as floatd
from optuna.distributions import IntDistribution as intd
from optuna.integration import OptunaSearchCV
from os import listdir, walk
from os.path import isfile, join
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping
from torch import nn
from torch import tensor
from tqdm import notebook

### Загрузка данных
<a id="section_1_2"></a>

Загружаем датасет, содержащий названия фильмов и соответствующих им уровней английского языка:

In [2]:
df = pd.read_excel("./data/English_level/English_scores/movies_labels.xlsx")

Получаем пути к файлам с субтитрами:

In [3]:
filenames_path = "./data/English_level/English_scores/Subtitles_all"
filenames = next(walk(filenames_path), ([], [], []))

Загружаем словари "The Oxford 5000™ by CEFR level (American English)" и "The Oxford 5000™ by CEFR level" (соответствие слов уровням знания английского языка):

In [4]:
level_path = './data/English_level/Oxford_CEFR_level'
files_level_list = [filepath for filepath in pathlib.Path(level_path).glob('**/*.pdf')]

In [5]:
text = []
current_level = None
words_by_level = {'A1' : [], 'A2' : [], 'B1' : [], 'B2' : [], 'C1' : []}
exclude_words = {'prep.', 'adv.', 'n.', 'adj.', 'v.', 'det.', 'conj.', 'indefinite', 'pron.',
                'auxiliary', 'modal', 'modal', 'det./pron./adv.', 'det./adj.', 'det./number',
                'number/det.', 'adj./pron.', 'adv./prep.', 'det./', 'conj./prep.', '©'}
for path in files_level_list:
    doc = fitz.open(path)
    for page in doc:
        blocks = page.get_text('dict')['blocks']
        for block in blocks:
            if 'lines' in block:
                for line in block['lines']:
                    for span in line['spans']:
                        text = span['text']
                        if text in ['A1', 'A2', 'B1', 'B2', 'C1']:
                            current_level = text
                        elif current_level:
                            try:
                                word = text.split()[0]
                            except:
                                pass
                            word = word.strip(' ,\'"')
                            word = re.sub(r'\d+$', '', word)
                            if word and word not in exclude_words and not word.startswith('('):
                                words_by_level[current_level].append(word)
words_by_level[current_level] = list(set(words_by_level[current_level]))
data = {'A1': words_by_level['A1'], 'A2': words_by_level['A2'], 'B1': words_by_level['B1'],
        'B2': words_by_level['B2'], 'C1': words_by_level['C1']}
word_level = pd.DataFrame.from_dict(data, orient='index').T
word_level.columns = map(str.lower, word_level.columns)

### Общая информация
<a id="section_1_3"></a>

Общая информация по датафрейму, содержащему названия фильмов и соответствующих им уровней английского языка:

In [6]:
print('Размер датафрейма:')
print(df.shape)
print('_' * 50)
print()
print('Общая информация о датафрейме:')
print(df.info(memory_usage='deep'))
print('_' * 50)
print()
print('Несколько строк датафрейма:')
display(df.sample(random_state=0, n=5))
print('_' * 50)
print()
print('Количество дубликатов:')
print(df['Movie'].duplicated().sum())
print('_' * 50)  
print()
print('Описательная статистика строковых значений:')
display(df.describe(include='object').T)
print('_' * 5)
print('Описательная статистика числовых значений:')
display(df.describe().T)
print('_' * 50)
print()
print('Количество пропусков:')
print(df.isna().sum())
print()
print('Количество пропусков в процентах:')
print(df.isna().mean()*100)  

Размер датафрейма:
(241, 3)
__________________________________________________

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      241 non-null    int64 
 1   Movie   241 non-null    object
 2   Level   241 non-null    object
dtypes: int64(1), object(2)
memory usage: 36.2 KB
None
__________________________________________________

Несколько строк датафрейма:


,id,Movie,Level
109,109,We_are_the_Millers(2013),B1
71,71,Ready_or_not(2019),A2/A2+
37,37,Groundhog_day(1993),B1
74,74,Soul(2020),B1
108,108,Warm_bodies(2013),B1


__________________________________________________

Количество дубликатов:
4
__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
Movie,241,237,Powder(1995),2
Level,241,7,B2,101


_____
Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
id,241.0,120.0,69.714896,0.0,60.0,120.0,180.0,240.0


__________________________________________________

Количество пропусков:
id       0
Movie    0
Level    0
dtype: int64

Количество пропусков в процентах:
id       0.0
Movie    0.0
Level    0.0
dtype: float64


Общая информация по датафрейму, содержащему данные словарей "The Oxford 5000™ by CEFR level (American English)" и "The Oxford 5000™ by CEFR level" (соответствие слов уровням знания английского языка):

In [7]:
word_level.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2881 entries, 0 to 2880
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a1      1923 non-null   object
 1   a2      1781 non-null   object
 2   b1      1630 non-null   object
 3   b2      2881 non-null   object
 4   c1      1369 non-null   object
dtypes: object(5)
memory usage: 112.7+ KB


**Вывод:**  
1. Загружены данные по названиям фильмов и соответствующих им уровней английского языка. Датафрейм состоит из 241-го наблюдения и 3-х признаков:
- "id" - идентификатор фильма (формат данных - int64);
- "Movie" - название фильма  (формат данных - object);
- "Level" - соответствующий уровень английского языка (формат данных - object).  

  Количество уникальных значений:
- "Movie" - 237;
- "Level" - 7.


  В датафрейме присутствуют 4 дубликата. Пропуски отсутствуют.  

2. Загружены данные из словарей "The Oxford 5000™ by CEFR level (American English)" и "The Oxford 5000™ by CEFR level" (соответствие слов уровням знания английского языка). Датафрейм состоит из 2881 строк и 5 признаков:
- "a1" - 1923 слова;
- "a2" - 1781 слово;
- "b1" - 1630 слов;
- "b2" - 2881 слово;
- "c1" - 1369 слов.

## Предобработка данных
<a id="section_2"></a>

Создаем функцию вывода информации по датафрейму после промежуточных преобразований:

In [9]:
def dataframe_info(df_info):
    print('Общая информация о датафрейме:')
    print(df.info(memory_usage='deep'))
    print('_' * 50)
    print()
    print('Несколько строк датафрейма:')
    display(df_info.sample(random_state=0, n=5))
    print('_' * 50)
    print()
    print('Описательная статистика строковых значений:')
    display(df.describe(include='object').T)
    print('_' * 5)
    print('Описательная статистика числовых значений:')
    display(df.describe().T)
    print('_' * 50)
    print()

### Названия признаков
<a id="section_2_1"></a>

Переводим "верблюжий" регистр названий столбцов в "змеиный":

In [10]:
def camel_to_snake(str):
    snake_register = ''
    for i in str:
        if i.isupper():
            snake_register += '_' + i.lower()
        else:
            snake_register += i
    return snake_register.lstrip('_')

In [11]:
for column in df.columns:
    column_new = camel_to_snake(column)
    df = df.rename(columns={column : column_new})

Выводим информацию по датафрейму:

In [12]:
dataframe_info(df)

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      241 non-null    int64 
 1   movie   241 non-null    object
 2   level   241 non-null    object
dtypes: int64(1), object(2)
memory usage: 36.2 KB
None
__________________________________________________

Несколько строк датафрейма:


,id,movie,level
109,109,We_are_the_Millers(2013),B1
71,71,Ready_or_not(2019),A2/A2+
37,37,Groundhog_day(1993),B1
74,74,Soul(2020),B1
108,108,Warm_bodies(2013),B1


__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
movie,241,237,Powder(1995),2
level,241,7,B2,101


_____
Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
id,241.0,120.0,69.714896,0.0,60.0,120.0,180.0,240.0


__________________________________________________



### Дубликаты
<a id="section_2_2"></a>

Удаляем дубликаты по названиям фильмов:

In [13]:
df = df.drop_duplicates(['movie']).reset_index(drop=True)

Выводим информацию по датафрейму:

In [14]:
dataframe_info(df)

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      237 non-null    int64 
 1   movie   237 non-null    object
 2   level   237 non-null    object
dtypes: int64(1), object(2)
memory usage: 35.6 KB
None
__________________________________________________

Несколько строк датафрейма:


,id,movie,level
173,177,Suits.S01E06.1080p.BluRay.AAC5.1.x265-DTG.02.EN,B2
152,156,Suits.Episode 1- Denial,B2
106,110,While_You_Were_Sleeping(1995),B1
111,115,The Walking Dead-S01E04-Vatos.English,A2
125,129,Seven.Worlds.One.Planet.S01E06.2160p.BluRay.Re...,B1


__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
movie,237,237,10_Cloverfield_lane(2016),1
level,237,7,B2,101


_____
Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
id,237.0,120.780591,69.990271,0.0,60.0,122.0,181.0,240.0


__________________________________________________



### Уровни английского языка
<a id="section_2_3"></a>

Выводим представленные в датафрейме уровни английского языка и количество фильмов, соответствующих каждому уровню:

In [15]:
def level_preprocessing(df_lvl):
    df_lvl = df_lvl['level'].value_counts().sort_index().reset_index()
    df_lvl['percent'] = (100 * df_lvl['count'] / df_lvl['count'].sum()).round(1)
    display(df_lvl)
#     display(df['level'].value_counts().sort_index().reset_index())

In [16]:
level_preprocessing(df)

,level,count,percent
0,A2,6,2.5
1,A2/A2+,26,11.0
2,"A2/A2+, B1",4,1.7
3,B1,52,21.9
4,"B1, B2",8,3.4
5,B2,101,42.6
6,C1,40,16.9


В датафрейме присутствуют промежуточные уровни "A2/A2+", "A2/A2+, B1", "B1, B2". Избавляемся от них, переопределяя уровни таким образом, чтобы фильмы промежуточных уровней добавлялись к граничным уровням, наименее представленным в датаврейме, чтобы уменьшить дисбаланс классов.  
Переназначаем уровни следующим образом:
- "A2/A2+" в "A2";
- "A2/A2+, B1" в "B1";
- "B1, B2" в "B1".

In [17]:
df['level'] = df['level'].replace({'A2/A2+' : 'A2', 'A2/A2+, B1' : 'B1', 'B1, B2' : 'B1'})

In [18]:
level_preprocessing(df)

,level,count,percent
0,A2,32,13.5
1,B1,64,27.0
2,B2,101,42.6
3,C1,40,16.9


В датафрейме осталось 4 класса: "A2", "B1", "B2" и "C1". Присутствует дисбаланс классов.

Выводим информацию по датафрейму:

In [19]:
dataframe_info(df)

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      237 non-null    int64 
 1   movie   237 non-null    object
 2   level   237 non-null    object
dtypes: int64(1), object(2)
memory usage: 35.5 KB
None
__________________________________________________

Несколько строк датафрейма:


,id,movie,level
173,177,Suits.S01E06.1080p.BluRay.AAC5.1.x265-DTG.02.EN,B2
152,156,Suits.Episode 1- Denial,B2
106,110,While_You_Were_Sleeping(1995),B1
111,115,The Walking Dead-S01E04-Vatos.English,A2
125,129,Seven.Worlds.One.Planet.S01E06.2160p.BluRay.Re...,B1


__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
movie,237,237,10_Cloverfield_lane(2016),1
level,237,4,B2,101


_____
Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
id,237.0,120.780591,69.990271,0.0,60.0,122.0,181.0,240.0


__________________________________________________



### Объединение датафреймов
<a id="section_2_4"></a>

Необходимо добавить субтитры в датафрейм, содержащий названия фильмов и уровни.

Собираем в список имен файлов:

In [20]:
onlyfiles = []
for i in filenames[1]:
    onlyfiles += [f for f in listdir(filenames[0]+'\\'+i) if isfile(join(filenames[0]+'\\'+i, f))]

Удаляем из названий фильмов лишние символы:

In [21]:
df['movie'] = df.apply(lambda x: ' '.join(re.split(r',|-| |\|/|_|!|&|\'|\(|\)|\+|:|\.', x['movie'])).lower().strip(), axis=1)

Эпизоды сериала "Suits" в датафрейме имеют различные маски названий, отчего могут возникнуть сложности при подборе файлов с субтитрами. Приводим их к единообразию:

In [22]:
def movie_name(text):
    word = text.split()
    if ('suits' in text) and ('episode' not in text):
        for i in range(1, len(word)):
            if word[i-1] == 'suits':
                text = text.split()[i-1] + " " + text.split()[i] + " EngSub"
    return text

In [23]:
df['movie'] = df.apply(lambda x: movie_name(x['movie']), axis=1)

Добавляем в датафрейм названия файлов:

In [24]:
out_dir = '.'

files_dict = {}
for filepath in pathlib.Path(filenames_path).glob('**/*.srt'):
    file_name = str(filepath).split('\\')[-1]
    files_dict[file_name] = filepath
df_sub = pd.DataFrame(files_dict.items(), columns=['file_name', 'file_path'])

In [25]:
df['file_name'] = df['movie'].swifter.apply(lambda x: (difflib.get_close_matches(x, df_sub['file_name'])[:1] or [None])[0])

Pandas Apply:   0%|          | 0/237 [00:00<?, ?it/s]

Добавляем в датафрейм пути к файлам субтитров:

In [26]:
df = df.merge(df_sub,
              how = 'left',
              left_on='file_name',
              right_on='file_name'
             )[['movie', 'level', 'file_path']]

Добавляем в датафрейм субтитры:

In [27]:
def detected_encoding(file_path):
    rawdata = open(file_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result['encoding']

In [28]:
def get_subs_text(path):
    encoding = detected_encoding(path)
    subs = pysrt.open(str(path), encoding=encoding)
    return ' '.join([sub.text for sub in subs])

df['subs'] = df['file_path'].swifter.apply(get_subs_text)

Pandas Apply:   0%|          | 0/237 [00:00<?, ?it/s]

Удаляем из датафрейма названия фильмов и пути к файлам с субтитрами:

In [29]:
df = df.drop(['movie', 'file_path'],axis=1)

Выводим информацию по датафрейму:

In [30]:
dataframe_info(df)

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   level   237 non-null    object
 1   subs    237 non-null    object
dtypes: object(2)
memory usage: 15.7 MB
None
__________________________________________________

Несколько строк датафрейма:


,level,subs
173,B2,"Harvey, I don't need a perp-walk\nor a front-p..."
152,B2,You're the most amazing woman\nI have ever met...
106,B1,"LUCY: <i>Okay, there are two things that</i>\n..."
111,A2,( birds chirping ) - What?\n- Nothing. It's no...
125,B1,One continent on our planet changes more drama...


__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
level,237,4,B2,101
subs,237,237,"<font color=""#ffff80""><b>Fixed & Synced by boz...",1


_____
Описательная статистика числовых значений:


,count,unique,top,freq
level,237,4,B2,101
subs,237,237,"<font color=""#ffff80""><b>Fixed & Synced by boz...",1


__________________________________________________



### Обработка субтитров
<a id="section_2_5"></a>

Очищаем субтитры от лишних символов:

In [31]:
HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Z ]+[\)\]]'
LETTERS = r'[^a-zA-Z\'.,!? ]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'
MARKS = r'[\']+'

In [32]:
def clean_subs(txt):
    txt = re.sub(HTML, ' ', txt)                   # html тэги меняем на пробел
    txt = re.sub(TAG, ' ', txt)                    # тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt)               # комменты меняем на пробел
    txt = ".".join(txt.split('.')[1:-1])           # удаляем первый и последний субтитр (обычно это реклама)   
    txt = re.sub(LETTERS, ' ', txt)                # все что не буквы меняем на пробел
    txt = re.sub(SPACES, r'\1', txt)               # повторяющиеся пробелы меняем на один пробел
    txt = re.sub(DOTS, r'.', txt)                  # многоточие меняем на точку
    txt = txt.replace("''", '')                    # удаляем пустые кавычки
    txt = txt.replace(" n't", "n't")               # удаляем пробелы в отрицаниях
    txt = txt.encode('ascii', 'ignore').decode()   # удаляем все что не ascii символы
    txt = txt.strip()                              # удаляем пробелы в начале и в конце текста
    return txt

In [33]:
df['subs'] = df.swifter.apply(lambda x: clean_subs(x['subs']), axis=1)

Pandas Apply:   0%|          | 0/237 [00:00<?, ?it/s]

Поскольку в датафрейме недостаточно наблюдений для полноценного обучения модели, разделяем каждое наблюдение на несколько, содержащих по 70 предложений:

In [34]:
def separate_symbol(txt):
    list_sent = []                                      # список для хранения токенов
    num_sentence = 70                                   # максимальное количество токенов в наблюдении
    for sentence in sent_tokenize(txt):                 # проходим циклом ро токенам
        list_sent.append(sentence)                      # добавляем токены в список
        if len(list_sent) % num_sentence == 0:          # после каждого максимального количества токенов в наблюдении
            list_sent.append('#')                       # добавляем символ, по которому будет производиться разделение
    txt = ' '.join(list_sent)                           # объединяем список в текст
    txt = re.sub(r'\s+(?=(?:[\#]))', r'', txt)
    return txt

In [35]:
df['subs'] = df.apply(lambda x: separate_symbol(x['subs']), axis=1)        # добавляем в субтитры знаки разделения
df = (                                                                     # разделяем субтитры
    df
    .set_index(['level'])                                                  # признак level переносится без изменений
    .apply(lambda x: x.str.split('#').explode())                           # разделяем субтитры по символу '#'
    .reset_index()
    )

Поскольку при разделении от субтитров могут остаться "хвосты" малого размера, которые могут восприниматься моделью как выбросы, оставляем для решения поставленной задачи только наблюдения,  50 и более предложений:

In [36]:
df['length'] = df.apply(lambda x: len(sent_tokenize(x['subs'])), axis=1)         # добавляем признак - количество токенов
df = df.loc[df['length'] >= 50].reset_index(drop=True)                           # удаляем наблюдения, в которых менее 50 токенов

Определяем процент субтитров каждого классов после генерации дополнительных признаков:

In [37]:
level_preprocessing(df)

,level,count,percent
0,A2,637,14.4
1,B1,1363,30.8
2,B2,1858,42.0
3,C1,566,12.8


**Вывод:**  
1. Объеденены датафреймы, содержащие названия фильмов и субтитров.  
2. Субтитры очищены от посторонних символов.  
3. Сгенерированы дополнительные признаки путем разделения субтитров. Количество признаков увеличено с 237 до 4424.

## Расчёт дополнительных признаков
<a id="section_3"></a>

### Признаки оценок качества текста субтитров
<a id="section_3_1"></a>

Создаем признаки оценки качества текста субтитров:

In [38]:
def text_stat(df):
    '''функция оценки качества текста субтитров'''
    
    # количество слов, присутствующих в тексте;
    df['words'] = df.apply(lambda x: textstat.lexicon_count(x['subs']), axis=1)
    # среднее количество слов в предложении;
    df['words'] = df['words'] / df['length']
    # показатель легкости чтения Флеша
    df['fre'] = df.apply(lambda x: textstat.flesch_reading_ease(x['subs']), axis=1)
    # показатель удобочитаемости , который оценивает годы обучения, необходимые для понимания написанного
    df['si'] = df.apply(lambda x: textstat.smog_index(x['subs']), axis=1)
    # формула уровня успеваемости Флеша-Кинкейда
    df['fkd'] = df.apply(lambda x: textstat.flesch_kincaid_grade(x['subs']), axis=1)
    # уровень оценки текста с помощью формулы Коулмана-Ляу
    df['cli'] = df.apply(lambda x: textstat.coleman_liau_index(x['subs']), axis=1)
    # автоматический индекс удобочитаемости
    df['ari'] = df.apply(lambda x: textstat.automated_readability_index(x['subs']), axis=1)
    # уровень обучения, используя формулу Нью-Дейла-Чолла
    df['dcrs'] = df.apply(lambda x: textstat.dale_chall_readability_score(x['subs']), axis=1)
    # коэффициент сложных слов
    df['dw'] = df.apply(lambda x: textstat.difficult_words(x['subs']), axis=1)
    # уровень оценки текста по формуле Linsear
    df['lwf'] = df.apply(lambda x: textstat.linsear_write_formula(x['subs']), axis=1)
    # индекс FOG
    df['gf'] = df.apply(lambda x: textstat.gunning_fog(x['subs']), axis=1)
    return df

In [39]:
df = text_stat(df)

Выводим информацию по датафрейму:

In [40]:
dataframe_info(df)

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   level   4424 non-null   object 
 1   subs    4424 non-null   object 
 2   length  4424 non-null   int64  
 3   words   4424 non-null   float64
 4   fre     4424 non-null   float64
 5   si      4424 non-null   float64
 6   fkd     4424 non-null   float64
 7   cli     4424 non-null   float64
 8   ari     4424 non-null   float64
 9   dcrs    4424 non-null   float64
 10  dw      4424 non-null   int64  
 11  lwf     4424 non-null   float64
 12  gf      4424 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 9.9 MB
None
__________________________________________________

Несколько строк датафрейма:


,level,subs,length,words,fre,si,fkd,cli,ari,dcrs,dw,lwf,gf
220,B2,. a handsome young prince. lived in a beautifu...,70,7.971429,86.71,6.9,3.7,5.54,4.2,6.65,50,5.111111,4.86
1332,B1,These things are a test of character. And I h...,69,4.710145,97.50,6.3,1.6,3.84,3.0,6.06,24,2.733333,4.19
3448,B2,Offered only to our most exclusive members. Yo...,70,5.214286,88.43,7.0,3.0,4.49,3.4,6.69,40,3.214286,4.96
2905,B1,"Ladies and gentlemen, I'm sure you've noticed...",69,4.637681,99.23,5.3,0.9,2.77,2.5,6.25,30,2.857143,2.90
841,B2,"Once they have that location, they bomb the v...",69,9.130435,94.96,6.9,2.5,4.90,4.2,6.92,54,7.833333,5.41


__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
level,4424,4,B2,1858
subs,4424,4424,"Enjoy The Flick BEN ON PHONE Michelle, please ...",1


_____
Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
length,4424.0,68.951401,1.091201,50.000000,69.000000,69.000000,69.000000,70.000000
words,4424.0,5.897678,1.794363,2.333333,4.652174,5.507246,6.800000,18.608696
fre,4424.0,95.470002,5.500522,62.680000,90.460000,97.090000,98.410000,108.800000
si,4424.0,6.182030,0.818341,3.100000,5.700000,6.200000,6.700000,11.200000
fkd,4424.0,1.886325,1.014171,-0.800000,1.200000,1.700000,2.500000,8.700000
cli,4424.0,3.179882,1.314047,-1.610000,2.330000,3.150000,3.960000,10.090000
ari,4424.0,2.568671,1.082527,-0.500000,1.900000,2.500000,3.100000,9.900000
dcrs,4424.0,6.179919,0.535996,0.960000,5.820000,6.130000,6.500000,9.260000
dw,4424.0,31.224005,15.361048,3.000000,21.000000,29.000000,38.000000,197.000000
lwf,4424.0,3.261656,1.628396,1.260870,2.437500,2.928571,3.666667,50.000000


__________________________________________________



### Доли слов различного уровня
<a id="section_3_2"></a>

Для определения в субтитрах доли слов по уровню сложности и доли слов по частям речи производим лемматизацию субтитров. Для этого загружаем модель "en_core_web_lg":

In [41]:
sp = spacy.load('en_core_web_lg')

Производим лемматизацию:

In [42]:
def lemmatisation(text):
    lemma_text = []                              # список для хранения лемматизированных слов
    for word in sp(text.lower()):                # проходим по субтитрам
        if word.lemma_.isalpha():                # если лемма слова - текст
            lemma_text.append(word.lemma_)       # добавляем лемму в список
    return ' '.join(lemma_text)                  # объединяем список лемм в текст

In [43]:
df['subtitles_mod'] = df.swifter.apply(lambda x: lemmatisation(x['subs']), axis=1)

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Определяем доли слов по уровню сложности:

In [44]:
def words_sort(text, level):
    n = 0                                       # инициализируем переменную количества слов
    for i in text.split():                      # проходим по лемматизированному тексту
        # если слово есть в словари  "The Oxford 5000™ by CEFR level (American English)" и "The Oxford 5000™ by CEFR level"
        if i in list(word_level[level]):
            n += 1                              # увеличиваем счетчик
    koef = n / len(text.split())                # считаем долю
    return koef

In [45]:
words_sort_list = {'a_1' : 'a1', 'a_2' : 'a2',                             # словарь уровней слов
                   'b_1' : 'b1', 'b_2' : 'b2', 'c_1' : 'c1'}
for key, value in words_sort_list.items():                                 # рассчитываем долю слов каждого уровня
    df[key] = df.swifter.apply(lambda x: words_sort(x['subtitles_mod'], value), axis=1)

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

### Доли слов по частям речи
<a id="section_3_3"></a>

Определяем доли слов по частям речи. Для этого создаем словарь название признака - часть речи:

In [46]:
word_form_dict = {}                                                   # словарь для хранение частей речи
def word_form_pos(df):
    for sen in df['subtitles_mod']:                                   # перебираем субтитры
        for word in sp(sen):                                          # перебираем слова в субтитрах
            word_form_dict['form_'+(word.pos_).lower()] = word.pos_   # определяем части речи, присутствующие в субтитрах
    return word_form_dict

In [47]:
word_form_dict = word_form_pos(df)

Добавляем в датафрейм доли слов по частям речи:

In [48]:
def word_form(text, form):
    n = 0                                        # инициализируем переменную количества слов
    for word in sp(text):                        # проходим по словам
        if word.pos_ == form:                    # если слово соответствует рассматриваемой части речи
            n += 1                               # увеличиваем счетчик
    return n / len(text)

In [49]:
for key, value in word_form_dict.items():                               # проходим по словарю частей речи
    df[key] = df.swifter.apply(lambda x: word_form(x['subtitles_mod'], value), axis=1)

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4424 [00:00<?, ?it/s]

Удаляем признаки "subtitles_mod", "length", которые использовались для feature engineering:

In [50]:
df = df.drop(['subtitles_mod', 'length'], axis=1)

Выводим информацию по датафрейму:

In [51]:
dataframe_info(df)

Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   level       4424 non-null   object 
 1   subs        4424 non-null   object 
 2   words       4424 non-null   float64
 3   fre         4424 non-null   float64
 4   si          4424 non-null   float64
 5   fkd         4424 non-null   float64
 6   cli         4424 non-null   float64
 7   ari         4424 non-null   float64
 8   dcrs        4424 non-null   float64
 9   dw          4424 non-null   int64  
 10  lwf         4424 non-null   float64
 11  gf          4424 non-null   float64
 12  a_1         4424 non-null   float64
 13  a_2         4424 non-null   float64
 14  b_1         4424 non-null   float64
 15  b_2         4424 non-null   float64
 16  c_1         4424 non-null   float64
 17  form_verb   4424 non-null   float64
 18  form_det    4424 non-null   float64
 

,level,subs,words,fre,si,fkd,cli,ari,dcrs,dw,...,form_part,form_adv,form_pron,form_adj,form_sconj,form_cconj,form_num,form_x,form_punct,form_sym
220,B2,. a handsome young prince. lived in a beautifu...,7.971429,86.71,6.9,3.7,5.54,4.2,6.65,50,...,0.007423,0.007423,0.026158,0.019088,0.005302,0.004949,0.001767,0.0,0.0,0.0
1332,B1,These things are a test of character. And I h...,4.710145,97.50,6.3,1.6,3.84,3.0,6.06,24,...,0.007486,0.008734,0.039925,0.007486,0.005614,0.002495,0.001248,0.0,0.0,0.0
3448,B2,Offered only to our most exclusive members. Yo...,5.214286,88.43,7.0,3.0,4.49,3.4,6.69,40,...,0.007151,0.008801,0.045105,0.009351,0.009901,0.002750,0.003850,0.0,0.0,0.0
2905,B1,"Ladies and gentlemen, I'm sure you've noticed...",4.637681,99.23,5.3,0.9,2.77,2.5,6.25,30,...,0.009446,0.008186,0.045970,0.005668,0.001889,0.003149,0.000000,0.0,0.0,0.0
841,B2,"Once they have that location, they bomb the v...",9.130435,94.96,6.9,2.5,4.90,4.2,6.92,54,...,0.009464,0.010410,0.040379,0.010410,0.005047,0.002524,0.001262,0.0,0.0,0.0


__________________________________________________

Описательная статистика строковых значений:


,count,unique,top,freq
level,4424,4,B2,1858
subs,4424,4424,"Enjoy The Flick BEN ON PHONE Michelle, please ...",1


_____
Описательная статистика числовых значений:


,count,mean,std,min,25%,50%,75%,max
words,4424.0,5.897678,1.794363,2.333333,4.652174,5.507246,6.800000,18.608696
fre,4424.0,95.470002,5.500522,62.680000,90.460000,97.090000,98.410000,108.800000
si,4424.0,6.182030,0.818341,3.100000,5.700000,6.200000,6.700000,11.200000
fkd,4424.0,1.886325,1.014171,-0.800000,1.200000,1.700000,2.500000,8.700000
cli,4424.0,3.179882,1.314047,-1.610000,2.330000,3.150000,3.960000,10.090000
ari,4424.0,2.568671,1.082527,-0.500000,1.900000,2.500000,3.100000,9.900000
dcrs,4424.0,6.179919,0.535996,0.960000,5.820000,6.130000,6.500000,9.260000
dw,4424.0,31.224005,15.361048,3.000000,21.000000,29.000000,38.000000,197.000000
lwf,4424.0,3.261656,1.628396,1.260870,2.437500,2.928571,3.666667,50.000000
gf,4424.0,4.112848,0.888035,1.920000,3.500000,4.020000,4.610000,10.060000


__________________________________________________



**Вывод:**  
В датарейм добавлены следующие признаки:  
- words - среднее количество слов в предложении;
- fre - показатель легкости чтения Флеша;
- si - показатель удобочитаемости , который оценивает годы обучения, необходимые для понимания написанного;
- fkd - формула уровня успеваемости Флеша-Кинкейда;
- cli - уровень оценки текста с помощью формулы Коулмана-Ляу;
- ari - автоматический индекс удобочитаемости;
- dcrs - уровень обучения, используя формулу Нью-Дейла-Чолла;
- dw - коэффициент сложных слов;
- lwf -  уровень оценки текста по формуле Linsear;
- gf - индекс FOG;
- a_1, a_2, b_1, b_2, c_1 - доля слов соответствующего уровня;
- Признаки долей частей речи в субтитрах.

## Обучение нейросети
<a id="section_4"></a>

### Разделение выборки на обучающую и тестовую
<a id="section_4_1"></a>

Задаем параметр random_state:

In [52]:
RS = 42

Выделяем из выборки целевую переменную:

In [53]:
features = df.drop(['level'], axis=1)
target = df['level']

Разделение выборки на обучающую и тестовую. Поскольку обучение будет происходить с применением кросс-валидации, валидационная выборка не требуется.

In [54]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.3, stratify=target, random_state=RS)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

В исходном датафрейме целевая переменная имеет текстовый тип. Закодируем целевую переменную обучающей выборки с помощью OrdinalEncoder, результат преобразуем в тензор.

In [55]:
def ord_encoder(y):
    encoder = OrdinalEncoder()
    y = y.reset_index()
    y = encoder.fit_transform(y[['level']])
    y = pd.DataFrame(data=y)
    y = tensor(np.array(y), dtype=torch.long).reshape(-1)
    return y, encoder

In [56]:
y_train, encoder = ord_encoder(y_train)

### Балансировка классов
<a id="section_4_2"></a>

Рассчитываем веса классов:

In [57]:
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(pd.DataFrame(y_train.numpy())),
    y=pd.DataFrame(y_train.numpy()).values.reshape(-1)
)
class_weights = tensor(np.array(weights), dtype=torch.float32)

### Векторизация текста
<a id="section_4_3"></a>

Загружаем предобученную модель BERT и автотокенизатор. Поскольку векторизация будет производиться с учетои регистра, загружаем модель "bert-base-uncased":

In [58]:
model = ppb.AutoModel.from_pretrained('bert-base-uncased')
tokenizer = ppb.AutoTokenizer.from_pretrained('bert-base-uncased')

Задаем обучение на GPU, если такая возможность имеется:

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Задаем условия стратификации и перемешивания данных при кросс-валидации: 

In [60]:
model_dict = {}                                                             # словарь, для добавления в него лучших моделей
n_counts = 4                                                                # количество подгрупп при кросс-валидации
skf = StratifiedKFold(n_splits=n_counts, shuffle = True, random_state=RS)   # параметры стратификации/кросс-валидации

Векторизтруем текст:

In [99]:
class subs_vectorization(BaseEstimator, TransformerMixin):
    '''Класс обучения и трансформации датафреймов для обработки пропусков значений признаков "make" и "transmission"'''
    
    def fit(self, X, y=None):
        '''функция обучения'''
        X = X.copy()                                                               # делаем копию датафрейма входных признаков
        X_stat = X.drop(['subs'], axis=1)                                          # выделяем полученные ранее признаки
        tokenized = X['subs'].apply(                                               # токенизируем субтитры
            lambda x: tokenizer.encode(x, padding=True, truncation=True, max_length=512, add_special_tokens = True))
       # инициализируем переменную максимальной длины эмбендинга
        max_len = 0
        for i in tokenized.values:                                                 # определяем максимальную длину признака
            if len(i) > max_len:
                max_len = len(i)
        self.max_len = max_len
        
        scaler = StandardScaler()                                                  # инициализируем StandardScaler
        self.X_scaler = scaler.fit(X_stat)                                         # обучаем StandardScaler
        return self

    def transform(self, X):
        '''функция трансформации'''
        X = X.copy()                                                   # делаем копию датафрейма входных признаков
        X_stat = X.drop(['subs'], axis=1)                              # выделяем полученные ранее признаки
        MAX_LENGTH = self.max_len                                      # задаем максимальную длину эмбеддинга (получена в функции fit)
        tokenized = X['subs'].apply(                                   # токенизируем субтитры
            lambda x: tokenizer.encode(x, padding=True, truncation=True, max_length=MAX_LENGTH, add_special_tokens = True))
        padded = np.array([i + [0]*(MAX_LENGTH - len(i)) for i in tokenized.values])   # увеличиваем длину признаков до максимальной
        attention_mask = np.where(padded != 0, 1, 0)                                   # указываем нулевые и не нулевые значения
        batch_size = 128                                                                           # размер батча
        embeddings = []                                                                            # список для векторов признаков
        for i in notebook.tqdm(range(padded.shape[0] // batch_size)):                              # выделяем батчи из выборки
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])                        # преобразовываем данные в формат тензоров
            attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]) # преобразовываем данные в формат тензоров
            batch_embeddings = model(batch, attention_mask=attention_mask_batch.to(device))        # получаем эмбеддинги
            embeddings.append(batch_embeddings[0][:,0,:].detach().cpu().numpy())                   # добавляем батчи в список
            i_last = i                                                                             # индекс последней строки батча
        # обработка остатка, не попавшего в цикл
        batch = torch.LongTensor(padded[batch_size*(i_last+1):X.shape[0]])                        # преобразовываем данные в формат тензоров
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*(i_last+1):X.shape[0]]) # преобразовываем данные в формат тензоров
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))  # получаем эмбеддинги для батча
        embeddings.append(batch_embeddings[0][:,0,:].detach().cpu().numpy())                        # добавляем батчи в список
        X = np.concatenate(embeddings)                                                              # объединяем эмбеддинги
        X = pd.DataFrame(X)                                                                         # создаем датафрейм
        # производим нормализацию признаков, полученных до векторизации субтитров
        X_stat = self.X_scaler.transform(X_stat)
        # объединяем признаки, полученных до векторизации субтитров, в датафрейм
        X_stat = pd.DataFrame(X_stat)
        # объединяем все признаки в общий датафрейм
        X = pd.merge(X, X_stat, left_index=True, right_index=True)
        # преобразовываем датафрейм в тензор
        X = tensor(np.array(X), dtype=torch.float32)
        return X

### Обучение
<a id="section_4_4"></a>

Задаем архитектуру нейросети - полносвязная нейронная сеть с batchnormalization после промежуточных слоев:

In [100]:
class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_out_neurons, activation_1, activation_2, activation_3):
        super().__init__()
        activation_dict = {'ReLU' : nn.ReLU(), 'LeakyReLU' : nn.LeakyReLU()}
        
        self.dense0 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.nonlin0 = activation_dict.get(activation_1)                                            
        self.bn0 = nn.BatchNorm1d(n_hidden_neurons_1)
        self.dense1 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.nonlin1 = activation_dict.get(activation_2) 
        self.bn1 = nn.BatchNorm1d(n_hidden_neurons_2)
        self.dense2 = nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3)
        self.nonlin2 = activation_dict.get(activation_3) 
        self.bn2 = nn.BatchNorm1d(n_hidden_neurons_3)
        self.output = nn.Linear(n_hidden_neurons_3, n_out_neurons)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X, **kwargs):
        X = self.bn0(self.nonlin0(self.dense0(X)))
        X = self.bn1(self.nonlin1(self.dense1(X)))
        X = self.bn2(self.nonlin2(self.dense2(X)))
        X = self.softmax(self.output(X))
        return X

Инициализируем модель NeuralNetClassifier:

In [101]:
skorch_classifier = NeuralNetClassifier(
    Net,
    optimizer=torch.optim.Adam, 
    batch_size=128,
    max_epochs=200000,
    iterator_train__shuffle=True,
    criterion = nn.CrossEntropyLoss,
    criterion__weight=class_weights,
    verbose=False,
    callbacks=[EarlyStopping(patience=200, monitor='valid_loss')],
)

Создаем пайплайн для обучения модели:

In [102]:
pipeline = Pipeline(steps=[('subs_vectorization', subs_vectorization()),
                           ('model', skorch_classifier)])

Вычисляем размер входного и выходного тензоров:

In [103]:
pipeline_n_in_neurons = Pipeline(steps=[('subs_vectorization', subs_vectorization())])
X_train_pred = pipeline_n_in_neurons.fit_transform(X_train)                              
value_n_in_neurons = X_train_pred.shape[1]                                               # количество ядер на входном слое
value_n_out_neurons = len(y_train.unique())                                              # количество ядер на выходном слое

  0%|          | 0/24 [00:00<?, ?it/s]

Задаем сетку подбора гиперпараметров:

In [104]:
params_nn = {
    'lr' : floatd(0.00001, 0.00001, False, 0.1),
    'module__n_in_neurons' : intd(value_n_in_neurons, value_n_in_neurons, False, 1),
    'module__n_hidden_neurons_1': intd(4096, 8192, False, 4096),
    'module__n_hidden_neurons_2': intd(1024, 2048, False, 1024),
    'module__n_hidden_neurons_3': intd(128, 256, False, 128),
    'module__n_out_neurons' : intd(value_n_out_neurons, value_n_out_neurons, False, 1),
    'module__activation_1' : catd(['ReLU', 'LeakyReLU']),
    'module__activation_2' : catd(['ReLU', 'LeakyReLU']),
    'module__activation_3' : catd(['ReLU', 'LeakyReLU'])
}


new_params = {'model__' + key: params_nn[key] for key in params_nn}

Обучаем модель с помощью OptunaSearchCV. Для сокращения времени обучения назначаем параметр n_trials=1.  
Поскольку классификация многоклассовая, с дисбалансом, за метрику качества принимаем "f1_macro".

In [106]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
model_nn = OptunaSearchCV(pipeline,
                          new_params,
                          scoring="f1_macro",
                          cv=skf,
                          n_trials=1,
                          verbose=False)
model_nn.fit(X_train, y_train)

C:\Users\bond_\AppData\Local\Temp\ipykernel_31580\16349279.py:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  model_nn = OptunaSearchCV(pipeline,


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

OptunaSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=True),
               estimator=Pipeline(steps=[('subs_vectorization',
                                          subs_vectorization()),
                                         ('model',
                                          <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.Net'>,
))]),
               n_jobs=1, n_trials=1,
               param_distributions={'model__lr': FloatDistribution(high=1e-05, log=False, low=1e-0...
                                    'model__module__n_hidden_neurons_2': IntDistribution(high=2048, log=False, low=1024, step=1024),
                                    'model__module__n_hidden_neurons_3': IntDistribution(high=256, log=False, low=128, step=128),
                                    'model__module__n_in_neurons': IntDistribution(high=800, log=False, low=800, step=1),
                                    'model__module__n_out_neurons': IntDistribution(high=4, log=False, low=4, step=1)},
               scoring='f1_macro', verbose=False)

In [107]:
print('Значение f1_macro для лучшей модели на кросс валидации - {}'.format(model_nn.best_score_))
print('')
print('Параметры лучшей модели:')
model_nn.best_params_

Значение f1_macro для лучшей модели на кросс валидации - 0.6021479122644315

Параметры лучшей модели:


{'model__lr': 1e-05,
 'model__module__n_in_neurons': 800,
 'model__module__n_hidden_neurons_1': 4096,
 'model__module__n_hidden_neurons_2': 2048,
 'model__module__n_hidden_neurons_3': 256,
 'model__module__n_out_neurons': 4,
 'model__module__activation_1': 'ReLU',
 'model__module__activation_2': 'ReLU',
 'model__module__activation_3': 'ReLU'}

**Вывод:**
Обучена модель NeuralNetClassifier. Целевая метрика - f1_macro.  
Значение f1_macro для лучшей модели на кросс валидации - 0.60.  
Параметры лучшей модели:  
- lr: 1e-05;
- n_in_neurons: 800;
- n_hidden_neurons_1: 4096;
- n_hidden_neurons_2: 2048;
- n_hidden_neurons_3: 256;
- n_out_neurons: 4;
- activation_1: ReLU;
- activation_2: ReLU;
- activation_3: ReLU;

## Оценка качества модели
<a id="section_5"></a>

### Оценка на тестовой выборке
<a id="section_5_1"></a>

Рассчитываем значение метрики f1_macro на тестовой выборке:

In [108]:
pred_test = model_nn.best_estimator_.predict(X_test)

  0%|          | 0/10 [00:00<?, ?it/s]

Преобразовываем результаты из тензора в серию текстовых значений:

In [109]:
pred_dict = {}
for i in zip(pred_test.tolist(), encoder.inverse_transform(pred_test.reshape(-1, 1))[:][:, 0].tolist()):
    pred_dict[i[0]] = i[1]
    
pred_list = []
for i in pred_test:
    pred_list.append(pred_dict.get(i))
pred_test = pd.Series(pred_list)

Делаем предсказание на тестовой выборке:

In [110]:
f1_test = f1_score(y_test, pred_test, average='macro')
print('Значение f1_macro на тестовой выборке - {}.'.format(f1_test))

Значение f1_macro на тестовой выборке - 0.6306559809280587.


### Матрица ошибок
<a id="section_5_2"></a>

Выводим матрицу ошибок:

In [111]:
pd.crosstab(y_test, pred_test, rownames=['Actual'], colnames=['Predicted'])

Predicted,A2,B1,B2,C1
Actual,,,,
A2,122,33,31,5
B1,54,263,72,20
B2,33,83,400,42
C1,11,10,58,91


Выводим значения метрик по классам:

In [112]:
print(metrics.classification_report(y_test, pd.Series(pred_list), digits=3))

              precision    recall  f1-score   support

          A2      0.555     0.639     0.594       191
          B1      0.676     0.643     0.659       409
          B2      0.713     0.717     0.715       558
          C1      0.576     0.535     0.555       170

    accuracy                          0.660      1328
   macro avg      0.630     0.633     0.631      1328
weighted avg      0.661     0.660     0.660      1328



**Вывод:**  
1. Значение f1_macro на тестовой выборке - 0.63.
2. Наибольшая метрика f1 - при определении фильмов уровня B2 (наиболее представленных в выборке) - 0.715.
3. Наименьшая метрика f1 - при определении фильмов уровня C1 (наименее представленных в выборке) - 0.555.
4. В матрице ошибок чем дальше от главной диагонали, тем меньше предсказаний. То есть на ошибки в основном приходятся предсказания смежных классов, что в рамках поставленной задачи не представляется критическим недостатком так как:
- разметка данных проводилась вручную, отчего принадлежность фильма к определенному уровню сложности определена субъективными мнениями экспертов, каковые (мнения), могут различаться.
- человек, владеющий определенным уровнем английского языка с высокой долей вероятности способен воспринимать смежные уровни.
5. Основная проблема в прогнозировании - недостаток данных. Разделение субтитров на несколько наблюдений позволило увеличить размер выборки, но при этом была нарушена структура текста, что для BERT является негативным фактором.

## Общий вывод
<a id="section_6"></a>

**Импорт библиотек общая информация**  
 
Датафрейм, содержащий названия фильмов и категории:  
1. Загружены данные по названиям фильмов и соответствующих им уровней английского языка. Датафрейм состоит из 241-го наблюдения и 3-х признаков:
- "id" - идентификатор фильма (формат данных - int64);
- "Movie" - название фильма  (формат данных - object);
- "Level" - соответствующий уровень английского языка (формат данных - object).  

  Количество уникальных значений:
- "Movie" - 237;
- "Level" - 7.


  В датафрейме присутствуют 4 дубликата. Пропуски отсутствуют.  

2. Загружены данные из словарей "The Oxford 5000™ by CEFR level (American English)" и "The Oxford 5000™ by CEFR level" (соответствие слов уровням знания английского языка). Датафрейм состоит из 2881 строк и 5 признаков:
- "a1" - 1923 слова;
- "a2" - 1781 слово;
- "b1" - 1630 слов;
- "b2" - 2881 слово;
- "c1" - 1369 слов.


**Предобработка данных**  

1. Объеденены датафреймы, содержащие названия фильмов и субтитров.  
2. Субтитры очищены от посторонних символов.  
3. Сгенерированы дополнительные признаки путем разделения субтитров. Количество признаков увеличено с 237 до 4424.  


**Расчёт дополнительных признаков**  

В датарейм добавлены следующие признаки:  
- words - среднее количество слов в предложении;
- fre - показатель легкости чтения Флеша;
- si - показатель удобочитаемости , который оценивает годы обучения, необходимые для понимания написанного;
- fkd - формула уровня успеваемости Флеша-Кинкейда;
- cli - уровень оценки текста с помощью формулы Коулмана-Ляу;
- ari - автоматический индекс удобочитаемости;
- dcrs - уровень обучения, используя формулу Нью-Дейла-Чолла;
- dw - коэффициент сложных слов;
- lwf -  уровень оценки текста по формуле Linsear;
- gf - индекс FOG;
- a_1, a_2, b_1, b_2, c_1 - доля слов соответствующего уровня;
- Признаки долей частей речи в субтитрах.  


**Обучение нейросети**  

Обучена модель NeuralNetClassifier. Целевая метрика - f1_macro.  
Значение f1_macro для лучшей модели на кросс валидации - 0.60.  
Параметры лучшей модели:  
- lr: 1e-05;
- n_in_neurons: 800;
- n_hidden_neurons_1: 4096;
- n_hidden_neurons_2: 2048;
- n_hidden_neurons_3: 256;
- n_out_neurons: 4;
- activation_1: ReLU;
- activation_2: ReLU;
- activation_3: ReLU;  


**Оценка качества модели**  

1. Значение f1_macro на тестовой выборке - 0.63.
2. Наибольшая метрика f1 - при определении фильмов уровня B2 (наиболее представленных в выборке) - 0.715.
3. Наименьшая метрика f1 - при определении фильмов уровня C1 (наименее представленных в выборке) - 0.555.
4. В матрице ошибок чем дальше от главной диагонали, тем меньше предсказаний. То есть на ошибки в основном приходятся предсказания смежных классов, что в рамках поставленной задачи не представляется критическим недостатком так как:
- разметка данных проводилась вручную, отчего принадлежность фильма к определенному уровню сложности определена субъективными мнениями экспертов, каковые (мнения), могут различаться.
- человек, владеющий определенным уровнем английского языка с высокой долей вероятности способен воспринимать смежные уровни.
5. Основная проблема в прогнозировании - недостаток данных. Разделение субтитров на несколько наблюдений позволило увеличить размер выборки, но при этом была нарушена структура текста, что для BERT является негативным фактором.